import

In [81]:
import numpy as np
import sys,os
import random
import matplotlib.pyplot as plt
import csv
import ast
from collections import OrderedDict

Input Data 생성

x, y, z 데이터 중 

x가 0~99이면    정답 => [1,0,0]

x가 100~199이면 정답 => [0,1,0]

x가 200~299이면 정답 => [0,0,1]

In [88]:
input_data=np.empty((0,3))
t=np.empty((0,3))
for i in range(1100):
    input_data=np.append(input_data, np.array([[random.randrange(0,300) for j in range(3)]]),axis=0)
    x=input_data[-1,0]
    if x<100:
        t=np.append(t,np.array([[1, 0, 0]]),axis=0)
    elif x<200:
        t=np.append(t,np.array([[0, 1, 0]]),axis=0)
    elif x<300:
        t=np.append(t,np.array([[0, 0, 1]]),axis=0)
print(input_data.shape)

(1100, 3)


Input Data Normalize  (0~1)

정규화 식


<img src = "../simple_network/Data/normalize.png" width = "200" height = "75">



In [90]:
input_max=299
input_min=0
de=input_max-input_min
normalize_input=np.zeros((1100,3),dtype=float)
for i in range(1100):
    for j in range(3):
        normalize_input[i,j]=(input_data[i,j]-input_min)/de
print(input_data)
print(normalize_input)

[[ 47.  38. 185.]
 [178. 218. 166.]
 [287. 186. 281.]
 ...
 [ 86. 205. 253.]
 [ 82.  65. 242.]
 [ 59. 212.  88.]]
[[0.15719064 0.1270903  0.6187291 ]
 [0.59531773 0.72909699 0.55518395]
 [0.95986622 0.62207358 0.93979933]
 ...
 [0.28762542 0.68561873 0.84615385]
 [0.27424749 0.2173913  0.80936455]
 [0.19732441 0.7090301  0.29431438]]


Input Data 파일에 저장

In [68]:
np.savez("../simple_network/Data/input_data_set.npz", input_data)
np.savez("../simple_network/Data/t_set.npz", t)

Input Data 파일 읽기 테스트

np.savez()를 사용하여 numpy array 형 변수 읽어오기


In [69]:
read_input=np.load("../simple_network/Data/input_data_set.npz", allow_pickle=True)
read_t=np.load("../simple_network/Data/t_set.npz", allow_pickle=True)
r_input_data=read_input["arr_0"]
r_t_data=read_t["arr_0"]
print(r_input_data)
print(r_t_data)

[[ 38. 248. 180.]
 [294. 221.  38.]
 [170. 215. 246.]
 ...
 [271.  56.  17.]
 [ 27. 111. 110.]
 [224. 281.  72.]]
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]


가중치 W, 편향 B  생성

In [70]:
weight_init_std=0.01
params={}
params['W1']=weight_init_std*np.random.randn(3,4)
params['B1']=np.zeros(4)
params['W2']=weight_init_std*np.random.randn(4,4)
params['B2']=np.zeros(4)
params['W3']=weight_init_std*np.random.randn(4,3)
params['B3']=np.zeros(3)

가중치 W, 편향 B weight_bias.npz 파일에 저장

np.savez() --> 여러개의 배열을 1개의 압축되지 않은 *.npz 포맷 파일로 저장하기 (Save several arrays into a single file in uncompressed .npz format)

In [71]:
np.savez("../simple_network/Data/weight_bias.npz", **params)

dictionary형 변수 csv파일에 저장

In [80]:
# open a file in write mode
with open('../simple_network/Data/weight_bias.csv', 'w', newline='') as f:
    # create a csv writer
    writer = csv.writer(f)
    # write the keys of the dictionary as the header row
    writer.writerow(params.keys())
    # write the values of the dictionary as a single row
    writer.writerow(params.values())


npz 변수 파일 불러오기

np.load("파일명",allow_pickle=True)

In [ ]:
my_dictionary = np.load("../simple_network/Data/weight_bias.npz", allow_pickle=True)
paaa=dict(my_dictionary)
print(paaa)
print(params)

{'W1': array([[ 0.00100129,  0.00171493, -0.00744984, -0.0072463 ],
       [-0.00279707,  0.00038611,  0.00059763,  0.00212727],
       [ 0.01015244,  0.00896603,  0.01394524, -0.0116384 ]]), 'B1': array([0., 0., 0., 0.]), 'W2': array([[-0.01714785, -0.00084621,  0.01958009, -0.00482689],
       [-0.01493078, -0.00666829,  0.0004003 ,  0.00723725],
       [ 0.00795616,  0.00925017, -0.00475119, -0.01221938],
       [-0.00141743, -0.00099722,  0.01345534, -0.0136539 ]]), 'B2': array([0., 0., 0., 0.]), 'W3': array([[-0.00407708,  0.01794706, -0.00590321],
       [ 0.00541239,  0.00931136, -0.01577636],
       [-0.00011345, -0.00815884,  0.02110812],
       [-0.00380722,  0.00679975, -0.01780934]]), 'B3': array([0., 0., 0.])}
{'W1': array([[ 0.00100129,  0.00171493, -0.00744984, -0.0072463 ],
       [-0.00279707,  0.00038611,  0.00059763,  0.00212727],
       [ 0.01015244,  0.00896603,  0.01394524, -0.0116384 ]]), 'B1': array([0., 0., 0., 0.]), 'W2': array([[-0.01714785, -0.00084621,  0.0